# Setup

## PAckage Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import regex as re
import time
import os
import gc

# ignore warnings (gets rid of Pandas copy warnings)
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

# NLP tools
import spacy
nlp = spacy.load("en_core_web_sm")
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os

## Notebook Functions

In [43]:
def get_user_ratings(username):
    
    user_path = re.sub("\s+", "+", username)
    
    path = "https://www.boardgamegeek.com/xmlapi2/collection?username="+user_path+"&rated=1&stats=1"
    start = time.time()# log the start time for this entry
    
    print(path)
    
    flag = False
    
    while flag == False:
        
        print("Retrieving page")
        
        page = requests.get(path) # get the page
        game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup        
                
        if game_page.find('errors') != None:
            print("Invalid username")
            break
        
        else: 
            try:
                print(int(game_page.find('items')['totalitems']))
                flag = True
                continue
            except:
                print("failed, pausing")
                time.sleep(1)
                continue
        
        #try: 
        #    game_page.find('message').text == 'Invalid username specified'
        #    print("Invalid username")
        #    break
        #except: pass
        
        #try:
        #    print(int(game_page.find('items')['totalitems']))
        #    flag = True
        #    continue
        
        #except:
        #    print("failed, pausing")
        #    time.sleep(1.5)
        #    continue
    
    rated_items = game_page.find_all("item")
        
    game_ids = []
    game_ratings = []
    modified_record = []
    all_ratings = {}

    for game in rated_items:
        name = game.find("name").text
        game_id = game["objectid"] 
        rating = float(game.find("rating")["value"])
        date_rated = game.find("status")["lastmodified"]
        game_ids.append(game_id)
        game_ratings.append(rating)
        all_ratings[game_id]=rating
        modified = [username, game_id, rating, date_rated]
        modified_record.append(modified)

    raw_ratings_dict = {}
    raw_ratings_dict[username] = all_ratings
    
    user = pd.DataFrame.from_dict(raw_ratings_dict, orient="index")
    user_ratings_dates = pd.DataFrame(modified_record, columns=["Username", "BGGId", "Rating", "Date_rated"])
    
    time.sleep(.5)        
    
    return user, user_ratings_dates


In [3]:
def save_load_loop(users):
    
    ratings_dates = pd.read_pickle('userid/ratings_dates.pkl')
    user_ratings = pd.read_pickle('userid/user_ratings.pkl')

    print(users[:])
    
    for username in users:
    
        user, user_ratings_dates = get_user_ratings(username)
    
        ratings_dates = ratings_dates.append(user_ratings_dates)
        user_ratings = user_ratings.append(user)
        
        del user
        del user_ratings_dates
        gc.collect()
    
    float_conversion = user_ratings.astype('float')
    
    ratings_dates.to_pickle('userid/ratings_dates.pkl')
    float_conversion.to_pickle('userid/user_ratings.pkl')
    
    del ratings_dates
    del user_ratings
    gc.collect()

## Loading Comments

In [4]:
comments = pd.read_pickle('data_cleaned/comments.pkl')
comments

,BGGId,Name,Rating,Username,cleaned
0,174430,Gloomhaven,N/A,-Johnny-,con fiddliness time investment ridiculous mo...
1,174430,Gloomhaven,9,-mIDE-,kickstarter estimate delivery
2,174430,Gloomhaven,N/A,0 1 1 2 3 5 8,half good forge war
3,174430,Gloomhaven,10,0stuart0,good addictive coop game ve play desperately p...
4,174430,Gloomhaven,5,1 Family Meeple,imghttpscfgeekdostaticcommbsmbpngimg gloomh...
...,...,...,...,...,...
4172603,339592,Sheep in Disguise,10,Pitersenpai,este es el primer proyecto que patrocino en ki...
4172604,339592,Sheep in Disguise,10,Radalict,game look lot fun m look forward
4172605,339592,Sheep in Disguise,10,Rigoberto123,nt wait play support kickstarter subscribe mr ...
4172606,339592,Sheep in Disguise,10,TAB4two,wait game hour hilarious sheepnanigan family b...


In [5]:
all_users = sorted(list(comments['Username'].unique()))

del comments
gc.collect()

93

In [6]:
len(all_users)

182354

In [125]:
all_users.index('emeraldpack')

118900

## Get Ratings

In [128]:
sub_users = all_users[118900:]

In [129]:
start = 0

while True:
    
    end = start + 50
    users = sub_users[start:end]
    if len(users)==0: break
    save_load_loop(users)
    start += 50
    time.sleep(3)

['emeraldpack', 'emeraldsunflower', 'emerfob', 'emergencyoverride', 'emeric42', 'emerssso', 'emeryaj', 'emerys', 'emessi4h', 'emet84', 'emeterio', 'emezera', 'emfrank72', 'emgiees', 'emholland72', 'emichaels', 'emichaelsen52', 'emichaud10', 'emiem', 'emike', 'emike12', 'emil0', 'emil47', 'emilcavan', 'emildia', 'emilebro', 'emiliab', 'emilio72', 'emiliord75', 'emiliororo', 'emilizz1', 'emilo80', 'emilos', 'emilxf', 'emilyandclark', 'emilybess', 'emilyhancock', 'emilymork9', 'emilysaurus', 'emilysigler', 'emilyythestrange', 'eminb', 'eminga', 'emirikol2099', 'emirose', 'emitance', 'emixphi', 'emjay1950', 'emjayking', 'emjones']
https://www.boardgamegeek.com/xmlapi2/collection?username=emeraldpack&rated=1&stats=1
Retrieving page
3
https://www.boardgamegeek.com/xmlapi2/collection?username=emeraldsunflower&rated=1&stats=1
Retrieving page
17
https://www.boardgamegeek.com/xmlapi2/collection?username=emerfob&rated=1&stats=1
Retrieving page
2
https://www.boardgamegeek.com/xmlapi2/collection?us

failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=emmanuel+guardiola&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=emmanuel666&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
143
https://www.boardgamegeek.com/xmlapi2/collection?username=emmargarett&rated=1&stats=1
Retrieving page
0
https://www.boardgamegeek.com/xmlapi2/collection?username=emmatig&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
71
https://www.boardgamegeek.com/xmlapi2/collection?username=emmayo&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
16
https://www.boardgamegeek.com/xmlapi2/collection?username=emmechedue&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
37
https://www.boardgamegeek.com/xmlapi2/collection?username=emmepi14&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
158
https://www.boardgamegeek.com/xmlapi2/collect

failed, pausing
Retrieving page
failed, pausing
Retrieving page
657
https://www.boardgamegeek.com/xmlapi2/collection?username=emptyspaceexplorers&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=emptysubs&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
42
https://www.boardgamegeek.com/xmlapi2/collection?username=emptyvacuum&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
https://www.boardgamegeek.com/xmlapi2/collection?username=empy&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
36
https://www.boardgamegeek.com/xmlapi2/collection?username=emrahgoker&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
119
https://www.boardgamegeek.com/xmlapi2/collection?username=emreis&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
48
https://www.boardgamegeek.com/xmlapi2/collection?username=emremuderris&rated=1&stats=1
Retrieving page
failed, pausing

failed, pausing
Retrieving page
32
https://www.boardgamegeek.com/xmlapi2/collection?username=endless+nameless&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
397
https://www.boardgamegeek.com/xmlapi2/collection?username=endlsdremr&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
262
https://www.boardgamegeek.com/xmlapi2/collection?username=endoannie&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
2
https://www.boardgamegeek.com/xmlapi2/collection?username=endofcapitalism&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
10
https://www.boardgamegeek.com/xmlapi2/collection?username=endofunctor&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
44
https://www.boardgamegeek.com/xmlapi2/collection?username=endogame12&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
11
https://www.boardgamegeek.com/xmlapi2/collection?username=endormem&rated=1&stats=1
Retrieving page
failed, pausing
Retrieving page
1
http

KeyboardInterrupt: 

In [44]:
ratings_dates = pd.read_pickle('userid/ratings_dates.pkl')
user_ratings = pd.read_pickle('userid/user_ratings.pkl')

In [84]:
user_ratings

""


In [ ]:
ratings_dates

In [ ]:
break

##### ONLY RUN THIS TO RESET THE PICKLES ENTIRELY

In [126]:
ratings_dates = pd.DataFrame(columns=['Username', 'BGGId', 'Rating', 'Date_rated'])

user_ratings = pd.DataFrame()

In [127]:
ratings_dates.to_pickle('userid/ratings_dates.pkl')
user_ratings.to_pickle('userid/user_ratings.pkl')

In [ ]:
ratings_dates = pd.read_pickle('userid/ratings_dates.pkl')
user_ratings = pd.read_pickle('userid/user_ratings.pkl')

## Code Work

In [ ]:
break

In [ ]:
username = 'Shade92008'

In [23]:
path = "https://www.boardgamegeek.com/xmlapi2/collection?username=Shade92008&rated=1&stats=1"

In [20]:
path = "https://www.boardgamegeek.com/xmlapi2/collection?username=AK&rated=1&stats=1"

In [24]:
start = time.time()# log the start time for this entry

print(path)
    
page = requests.get(path) # get the page
game_page = BeautifulSoup(page.content, 'xml') # parse the page with beautifulsoup


https://www.boardgamegeek.com/xmlapi2/collection?username=Shade92008&rated=1&stats=1


In [27]:
game_page.find('errors') != None

False

In [48]:
game_page.find('message').text == 'Invalid username specified'

True

In [51]:
try: 
    game_page.find('message').text == 'Invalid username specified'
    print("is true")
except: pass

is true


In [ ]:
int(game_page.find('items')['totalitems']) > 0

In [ ]:
rated_items = game_page.find_all('item')
rated_items

In [ ]:
game_ids = []
game_ratings = []
modified_record = []
all_ratings = {}

for game in rated_items:
    name = game.find('name').text
    game_id = game['objectid'] 
    rating = game.find('rating')['value']
    date_rated = game.find('status')['lastmodified']
    print(game_id, rating, date_rated)
    game_ids.append(game_id)
    game_ratings.append(rating)
    all_ratings[game_id]=rating
    modified = [username, game_id, rating, date_rated]
    modified_record.append(modified)

raw_ratings_dict = {}


In [ ]:
modified_record

In [ ]:
raw_ratings_dict[username] = all_ratings
raw_ratings_dict

In [ ]:
user = pd.DataFrame.from_dict(raw_ratings_dict, orient='index')
user

In [ ]:
user_ratings_dates_test = pd.DataFrame(modified_record, columns=['Username', 'BGGId', 'Rating', 'Date_rated'])
user_ratings_dates_test

In [ ]:
ratings_dates_test = pd.DataFrame(columns=['Username', 'BGGId', 'Rating', 'Date_rated'])
ratings_dates_test

In [ ]:
ratings_dates_test = ratings_dates_test.append(user_ratings_dates_test)
ratings_dates_test

In [ ]:
def get_user_ratings(username):
    
    user_path = re.sub("\s+", "+", username)
    
    path = "https://www.boardgamegeek.com/xmlapi2/collection?username="+user_path+"&rated=1&stats=1"
    start = time.time()# log the start time for this entry

    print(path)
    
    page = requests.get(path) # get the page
    game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup
    
    
    try: 
        print(int(game_page.find('items')['totalitems']))
        
    except:
        
        print("failed, pausing")
        time.sleep(5)
        
        page = requests.get(path) # get the page
        game_page = BeautifulSoup(page.content, "xml") # parse the page with beautifulsoup
        print(int(game_page.find('items')['totalitems']))
    
    rated_items = game_page.find_all("item")
        
    game_ids = []
    game_ratings = []
    modified_record = []
    all_ratings = {}

    for game in rated_items:
        name = game.find("name").text
        game_id = game["objectid"] 
        rating = game.find("rating")["value"]
        date_rated = game.find("status")["lastmodified"]
        game_ids.append(game_id)
        game_ratings.append(rating)
        all_ratings[game_id]=rating
        modified = [username, game_id, rating, date_rated]
        modified_record.append(modified)

    raw_ratings_dict = {}
    raw_ratings_dict[username] = all_ratings
    
    user = pd.DataFrame.from_dict(raw_ratings_dict, orient="index")
    user_ratings_dates = pd.DataFrame(modified_record, columns=["Username", "BGGId", "Rating", "Date_rated"])
    
    time.sleep(.5)
    
    return user, user_ratings_dates
